# Fetch Challenge - Requirement 1

Review unstructured JSON data and diagram a new structured relational data model.

I will use the pandas library to analyze the data. My goal here is to:
- Get familiar with the data files one by one
- Prep them for a Data Model (remove duplicates, clean etc..)
- Start to think about how they relate to each other

With this knowledge, I'll be better-equipped to diagram it out, establish one-to-many, many-to-one relationships and all that.

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [3]:
# Read in the JSON files as pandas dataframes

users = pd.read_json('JSON-Files/users.json', lines=True)
brands = pd.read_json('JSON-Files/brands.json', lines=True)
receipts = pd.read_json('JSON-Files/receipts.json', lines=True)

## Users

Lets examine the Users dataframe first and answer the question: is it ready to be a table as is in our Data Model, or should we make changes?

In [4]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


I can tell there are duplicates IDs. We'll need to handle these before this is viable as a database table.

In [5]:
users_table = users.drop_duplicates(subset='_id')

I dont like how there are dictionaries for the fields: _id, createdDate and lastLogin. This is unclean and will make analysis tricky. We'll use the properties of Python dictionaries to extract the data we want (see next cell):

In [6]:
row = {'$oid': '5ff1e194b6a9d73a3a9f1052'}
row['$oid']

'5ff1e194b6a9d73a3a9f1052'

In [7]:
users_table['_id'] = users_table['_id'].apply(lambda x: x['$oid'])

In [8]:
users_table.head(2)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI


Kill two birds with one stone for the date fields. I found out that 1609687537858 is a millisecond timestamp from Google. I also noticed that there are missing values in the lastLogin column. We'll have to contruct our extract_date function to detect if the value is a dictionary first and not a NaN.

In [9]:
def extract_date(row):
    if isinstance(row, dict): # This line addresses a NaN error I got initially
        return pd.to_datetime(row['$date'], unit='ms').date()
    return row

In [10]:
users_table['createdDate'] = users_table['createdDate'].apply(extract_date)
users_table['lastLogin'] = users_table['lastLogin'].apply(extract_date)

In [11]:
users_table.head(3)

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03,2021-01-03,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03,2021-01-03,consumer,Email,WI
6,5ff1e1e8cfcf6c399c274ad9,True,2021-01-03,2021-01-03,consumer,Email,WI


## Brands

In [12]:
brands.head(3)

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176


Check for duplicate IDs.

In [13]:
brands['_id'].duplicated().sum()

0

There are no duplicates. However there are nested fields. Lets fix those and the table will be ready.

In [14]:
brands_table = brands.copy()

In [15]:
brands_table['_id'] = brands_table['_id'].apply(lambda x: x['$oid'])

In [16]:
brands_table['cpg'] = brands_table['cpg'].apply(lambda x: x['$id']['$oid'])

In [17]:
brands_table.head(3)

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,601ac114be37ce2ead437550,test brand @1612366101024,0.0,NaN
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,5332f5fbe4b03c9a25efd0ba,Starbucks,0.0,STARBUCKS
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,601ac142be37ce2ead437559,test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176


# Receipts

In [18]:
receipts.head(3)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b


In [19]:
receipts['_id'].duplicated().sum()

0

There are no duplicates. Lots of fields to clean up though. Also, note that rewardsReceiptItemList follows a new pattern. We'll revisit that one.

In [20]:
receipts_table = receipts.copy()

In [21]:
receipts_table['_id'] = receipts_table['_id'].apply(lambda x: x['$oid'])

In [22]:
date_columns = ['createDate','dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']

for col in date_columns:
    receipts_table[col] = receipts_table[col].apply(extract_date)

In [23]:
receipts_table.head(2)

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,500.0,2021-01-03,5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03,2021-01-03,2021-01-03,2021-01-03,2021-01-03,150.0,2021-01-02,2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052


In [53]:
receipts_table.shape

(1119, 15)

## A Look at rewardsReceiptItemList 

This column immidiately stands out becuase it contains multiple items/values. Despite this just seeming off to me, I did some reading and Youtubing and found that this property violates First Normal Form: you cant have repeating groups, you want one value per cell.

Also, with the data in this format it would be very difficult to answer queries down the road such has: rank the top 10 items by purchase count. I think we should turn this column into its own table.

In [25]:
receipts_table['rewardsReceiptItemList'].iloc[0]

[{'barcode': '4011',
  'description': 'ITEM NOT FOUND',
  'finalPrice': '26.00',
  'itemPrice': '26.00',
  'needsFetchReview': False,
  'partnerItemId': '1',
  'preventTargetGapPoints': True,
  'quantityPurchased': 5,
  'userFlaggedBarcode': '4011',
  'userFlaggedNewItem': True,
  'userFlaggedPrice': '26.00',
  'userFlaggedQuantity': 5}]

In [26]:
receipts_table['rewardsReceiptItemList'].iloc[12]

[{'barcode': '034100573065',
  'description': 'MILLER LITE 24 PACK 12OZ CAN',
  'finalPrice': '29',
  'itemPrice': '29',
  'partnerItemId': '1',
  'pointsEarned': '870.0',
  'pointsPayerId': '5332f709e4b03c9a25efd0f1',
  'quantityPurchased': 1,
  'rewardsGroup': 'MILLER LITE 24 PACK',
  'rewardsProductPartnerId': '5332f709e4b03c9a25efd0f1',
  'targetPrice': '77'},
 {'barcode': '034100573065',
  'description': 'MILLER LITE 24 PACK 12OZ CAN',
  'finalPrice': '29',
  'itemPrice': '29',
  'partnerItemId': '2',
  'pointsEarned': '870.0',
  'pointsPayerId': '5332f709e4b03c9a25efd0f1',
  'quantityPurchased': 1,
  'rewardsGroup': 'MILLER LITE 24 PACK',
  'rewardsProductPartnerId': '5332f709e4b03c9a25efd0f1',
  'targetPrice': '77'},
 {'barcode': '034100573065',
  'description': 'MILLER LITE 24 PACK 12OZ CAN',
  'finalPrice': '29',
  'itemPrice': '29',
  'partnerItemId': '3',
  'pointsEarned': '870.0',
  'pointsPayerId': '5332f709e4b03c9a25efd0f1',
  'quantityPurchased': 1,
  'rewardsGroup': 'MI

Wowser. So this field contains lists of lists where one list consitutes an item on the reciept. We can use 2 handy functions here:
- explode() to put each list on its own row
- json_normalize to make each value in the list its own column

This is how we will make this thing its own table.

In [47]:
receipt_items = receipts_table[['_id', 'rewardsReceiptItemList']].explode('rewardsReceiptItemList').reset_index(drop=True)

In [48]:
receipt_items.shape

(7381, 2)

In [49]:
receipt_items.head(2)

,_id,rewardsReceiptItemList
0,5ff1e1eb0a720f0523000575,"{'barcode': '4011', 'description': 'ITEM NOT F..."
1,5ff1e1bb0a720f052300056b,"{'barcode': '4011', 'description': 'ITEM NOT F..."


In [50]:
receipt_items_table = pd.concat([
    receipt_items['_id'].rename('receipt_id'),
    pd.json_normalize(receipt_items['rewardsReceiptItemList'])
], axis=1)

In [51]:
receipt_items_table.head(3)

,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# Give the table a primary key
receipt_item_table = receipt_items_table.reset_index().rename(columns={'index': '_id'})

I suspect that quite a few columns could be dropped from this table but to be honest, Im not sure how to methodically determine which of the 35 should go. I will keep them in here for now and if the time ever comes would love to dicuss this.

In [56]:
receipt_item_table.head(2)

,_id,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Lets now drop the rewardsReceiptItemList column from receipts_table and our tables will be good to go.

In [55]:
receipts_table.drop(['rewardsReceiptItemList'],axis=1,inplace=True)

## Data Model

**Users Table**
- Contains user information
- Primary key: user_id

**Brands Table**
- Contains brand information
- Primary key: brand_id

**Receipts Table**
- Contains receipt-level information
- Primary key: receipt_id
- Foreign key: user_id (links to Users)

**Receipt_Items Table**
- Contains unique item information
- Primary key: item_id
- Foreign keys:
    - receipt_id (links to Receipts)
    - barcode (links to Brands)

**Relationships**
- Users can have many Receipts
- Receipts can have many Items
- Brands can appear in many Items
- Each item references one brand and belongs to one Receipt 

We'll export these we can use them later in a SQL enviroment.

In [57]:
users_table.to_csv('users_table.csv', index=False)
brands_table.to_csv('brands_table.csv', index=False)
receipts_table.to_csv('receipts_table.csv', index=False)
receipt_item_table.to_csv('receipt_item_table.csv', index=False)